# Plan

* organizar preprocesamiento
 * corregir fecha TRM
* Dejar Solo funciones
* programar ciclos
* dejar registro quiebras
* dejar registro cuantos quiebran y sobreviven
* analizar resultados en otro script
 * hurst
 * ley potencia
* probar con nuevo resultado
* probar con otro indice
* fin


# Librerías y configuraciones generales

In [1]:
import pandas as pd
from random import randrange
import random
import math
from statsmodels.tsa.api import SimpleExpSmoothing
import matplotlib.pyplot as plt
import warnings

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

# Preprocesamiento 
* Carga datos de entrada

In [4]:
rango_lectura=['2020-10-09','2021-05-18']
rango_indice=['2020-09-09','2021-05-18']

In [5]:
base_rss = pd.read_csv (r'TranformacionBases_Java/Rss_adap_multi.csv', engine='python', encoding = "ISO-8859-1", sep=';')
base_twitter = pd.read_csv (r'TranformacionBases_Java/Twitter_adap_multi.csv', engine='python', encoding = "ISO-8859-1", sep=';')

In [6]:
# rangos datos lecturas
print("rango rss: min: ",min(base_rss['date_adj']), " max: ",max(base_rss['date_adj']))
print("rango twitter: min: ",min(base_twitter['date']), " max: ",max(base_twitter['date']))

rango rss: min:  2020-10-09  max:  2021-05-18
rango twitter: min:  2020-10-05  max:  2021-05-30


In [7]:
datos_twitter=base_twitter[['id_fuente','date','polaridad']].groupby(['id_fuente','date']).mean().reset_index()
datos_twitter

,id_fuente,date,polaridad
0,BluRadioCo,2020-10-09,-0.347026
1,BluRadioCo,2020-10-10,-0.271511
2,BluRadioCo,2020-10-11,-0.635847
3,BluRadioCo,2020-10-25,-0.249044
4,BluRadioCo,2020-10-26,-0.410109
...,...,...,...
4123,rcnradio,2021-05-26,-0.692930
4124,rcnradio,2021-05-27,-0.619450
4125,rcnradio,2021-05-28,-0.528466
4126,rcnradio,2021-05-29,-0.505510


In [8]:
datos_twitter['date'] = pd.to_datetime(datos_twitter['date'])

In [9]:
datos_rss=base_rss[['fuenteRss','date','polaridad']].groupby(['fuenteRss','date']).mean().reset_index()
datos_rss

,fuenteRss,date,polaridad
0,http://elnacional.com.do/feed/,1/11/2020 04:00,-0.986164
1,http://elnacional.com.do/feed/,1/11/2020 12:00,-0.975831
2,http://elnacional.com.do/feed/,1/11/2020 12:09,-0.105395
3,http://elnacional.com.do/feed/,1/11/2020 12:41,0.157199
4,http://elnacional.com.do/feed/,1/11/2020 12:44,-0.378361
...,...,...,...
110015,https://www.reforma.com/rss/portada.xml,9/5/2021 05:02,-0.989599
110016,https://www.reforma.com/rss/portada.xml,9/5/2021 11:01,-0.810466
110017,https://www.reforma.com/rss/portada.xml,9/5/2021 15:19,0.423649
110018,https://www.reforma.com/rss/portada.xml,9/5/2021 15:49,-0.641010


In [10]:
datos_rss['date'] = pd.to_datetime(datos_rss['date'])

In [11]:
##igualar rangos

In [12]:
datos_trm=pd.read_csv (r'TranformacionBases_Java/TRM_total_adap_multi.csv', engine='python', encoding = "ISO-8859-1", sep=';')

In [13]:
datos_trm=datos_trm[datos_trm['date']>=rango_indice[0]]
datos_trm=datos_trm[datos_trm['date']<=rango_indice[1]]

In [14]:
datos_trm

,Fecha,fecha operación,TRM,Diferencia,Dia sem,nom Dia Sem,fin Semana,Dif Cero,date
10515,10/9/2020,9/9/2020,3717.25,-39.96,4,mié,0,0,2020-09-09
10516,11/9/2020,10/9/2020,3700.28,-16.97,5,jue,0,0,2020-09-10
10517,12/9/2020,11/9/2020,3709.00,8.72,6,vie,0,0,2020-09-11
10518,13/9/2020,12/9/2020,3709.00,0.00,7,sáb,1,1,2020-09-12
10519,14/9/2020,13/9/2020,3709.00,0.00,1,dom,1,1,2020-09-13
...,...,...,...,...,...,...,...,...,...
10762,15/5/2021,14/5/2021,3682.84,-45.25,6,vie,0,0,2021-05-14
10763,16/5/2021,15/5/2021,3682.84,0.00,7,sáb,1,1,2021-05-15
10764,17/5/2021,16/5/2021,3682.84,0.00,1,dom,1,1,2021-05-16
10765,18/5/2021,17/5/2021,3682.84,0.00,2,lun,0,1,2021-05-17


In [15]:
datos_trm['date'] = pd.to_datetime(datos_trm['date'])

In [16]:
datos_twitter.columns=['fuente','date','polaridad']
datos_rss.columns=['fuente','date','polaridad']

In [17]:
datos_rss

,fuente,date,polaridad
0,http://elnacional.com.do/feed/,2020-01-11 04:00:00,-0.986164
1,http://elnacional.com.do/feed/,2020-01-11 12:00:00,-0.975831
2,http://elnacional.com.do/feed/,2020-01-11 12:09:00,-0.105395
3,http://elnacional.com.do/feed/,2020-01-11 12:41:00,0.157199
4,http://elnacional.com.do/feed/,2020-01-11 12:44:00,-0.378361
...,...,...,...
110015,https://www.reforma.com/rss/portada.xml,2021-09-05 05:02:00,-0.989599
110016,https://www.reforma.com/rss/portada.xml,2021-09-05 11:01:00,-0.810466
110017,https://www.reforma.com/rss/portada.xml,2021-09-05 15:19:00,0.423649
110018,https://www.reforma.com/rss/portada.xml,2021-09-05 15:49:00,-0.641010


In [18]:
datos_lectura=datos_twitter.append(datos_rss)

In [19]:
datos_lectura

,fuente,date,polaridad
0,BluRadioCo,2020-10-09 00:00:00,-0.347026
1,BluRadioCo,2020-10-10 00:00:00,-0.271511
2,BluRadioCo,2020-10-11 00:00:00,-0.635847
3,BluRadioCo,2020-10-25 00:00:00,-0.249044
4,BluRadioCo,2020-10-26 00:00:00,-0.410109
...,...,...,...
110015,https://www.reforma.com/rss/portada.xml,2021-09-05 05:02:00,-0.989599
110016,https://www.reforma.com/rss/portada.xml,2021-09-05 11:01:00,-0.810466
110017,https://www.reforma.com/rss/portada.xml,2021-09-05 15:19:00,0.423649
110018,https://www.reforma.com/rss/portada.xml,2021-09-05 15:49:00,-0.641010


In [20]:
datos_lectura=datos_lectura[datos_lectura['polaridad']!=0]

## Carga Maestro Fuentes

In [21]:
base_maestro_fuentes = pd.read_csv (r'TranformacionBases_Java/maestro_fuentes.csv', engine='python', encoding = "ISO-8859-1", sep=';')

In [22]:
base_maestro_fuentes.reset_index()

,index,fuente,tipo
0,0,http://elnacional.com.do/feed/,rss
1,1,http://ep00.epimg.net/rss/economia/portada.xml,rss
2,2,http://ep00.epimg.net/rss/tags/ultimas_noticia...,rss
3,3,http://feeds.nytimes.com/nyt/rss/Business,rss
4,4,http://feeds.nytimes.com/nyt/rss/Technology,rss
...,...,...,...
132,132,elcolombiano,twitter
133,133,elespectador,twitter
134,134,lafm,twitter
135,135,rcnmundo,twitter


In [23]:
datos_lectura=datos_lectura.merge(base_maestro_fuentes.reset_index(), left_on='fuente', right_on='fuente', how='left')
datos_lectura

,fuente,date,polaridad,index,tipo
0,BluRadioCo,2020-10-09 00:00:00,-0.347026,118,twitter
1,BluRadioCo,2020-10-10 00:00:00,-0.271511,118,twitter
2,BluRadioCo,2020-10-11 00:00:00,-0.635847,118,twitter
3,BluRadioCo,2020-10-25 00:00:00,-0.249044,118,twitter
4,BluRadioCo,2020-10-26 00:00:00,-0.410109,118,twitter
...,...,...,...,...,...
104999,https://www.reforma.com/rss/portada.xml,2021-09-05 05:02:00,-0.989599,117,rss
105000,https://www.reforma.com/rss/portada.xml,2021-09-05 11:01:00,-0.810466,117,rss
105001,https://www.reforma.com/rss/portada.xml,2021-09-05 15:19:00,0.423649,117,rss
105002,https://www.reforma.com/rss/portada.xml,2021-09-05 15:49:00,-0.641010,117,rss


## Calendario

In [24]:
calendario = pd.DataFrame({"Date": pd.date_range(rango_indice[0], rango_indice[1])})

In [25]:
calendario

,Date
0,2020-09-09
1,2020-09-10
2,2020-09-11
3,2020-09-12
4,2020-09-13
...,...
247,2021-05-14
248,2021-05-15
249,2021-05-16
250,2021-05-17


# Parametros Globales

In [26]:
#AGENTES
NUM_FUENTES_TOTATALES=137
NUM_FUENTES_INTERPRETACION=5

In [27]:
#UNIVERSO
POBLACION_INICIAL=100
CANT_COMPRADORES=0.7

#LIMITE_QUIEBRA=1000
LIMITE_QUIEBRA=10000000
MAX_GENERACION_AGENTES=0.2

POBLACION_ACTUAL=0

In [28]:
FECHA_PULSO=36   #fecha inicial con información suficiente

In [29]:
VALOR_MOMENTO_INDICE=datos_trm[datos_trm['date']<=calendario.iloc[FECHA_PULSO]['Date']].iloc[-1]['TRM']

In [30]:
VALOR_MOMENTO_INDICE

3854.47

In [31]:
DF_TRANSACCIONES=pd.DataFrame(columns=['idComprador','idVendedor','cantidadTransaccion','precioTransaccion','fechaPulso']) ###llevar antes al algoritmo

In [32]:
compradores=[]
vendedores=[]

# Agente

In [33]:
def crearAgente(id,tipo): #quizas falta el valor del var econ,
    agente={}
    agente['TIPO']=tipo
    agente['id']=id
    agente['edad']=0
    agente['capital']= randrange(1000,1000000000)
    agente['stockProducto']=randrange(10,1000000)
    agente['umbralStock']=randrange(10,1000000)
    agente['habilidadAdministrativa']=random.uniform(-0.02, 0.5)
    agente['tasaConsumo']=random.uniform(0.1, 0.5)
    if agente['TIPO']=='Comprador':
        agente['rentabilidad']=random.uniform(0.1, 0.2)
    elif agente['TIPO']=='Vendedor':
        agente['rentabilidad']=random.uniform(0.2, 0.6)
    agente['tamanoRegresion']= randrange(5,20)
    # perfilIntereses  No aplica
    agente['impactoTextos']=random.uniform(-1, 1)
    agente['estrategiaLectura']=[0]*NUM_FUENTES_INTERPRETACION
    agente['estrategiaLectura_imp']=[0]*NUM_FUENTES_INTERPRETACION
    for i in range(NUM_FUENTES_INTERPRETACION):
        agente['estrategiaLectura'][i]=randrange(NUM_FUENTES_TOTATALES)
        agente['estrategiaLectura_imp'][i]=random.random()
    agente['numeroDiasLecturas']=randrange(2,7)
    agente['umbralPronostico']=random.uniform(0.2, 0.5)
    agente['porcentajeCapitalCompra']=random.uniform(0, 1)
    agente['porcentajeStockVenta']=random.uniform(0, 1)
    #agente['costoMauc']  costo del commoditie según bolsa
    agente['costosFijos']=random.uniform(0.1, 0.3)
    agente['tasaProduccion']=random.uniform(0, 0.5)
    agente['limiteSobrePrecio']=random.uniform(0.01, 0.3)
    agente['viveza']=random.uniform(0.01, 0.2)
    agente['controlGastosEImpuestos']=random.uniform(0.01, 0.1)
    return agente

In [34]:
def pronosticoAgente(agente):    
    reg_inicio=FECHA_PULSO-agente['tamanoRegresion']
    rango_reg_agente=[calendario.iloc[reg_inicio]['Date'],calendario.iloc[FECHA_PULSO]['Date']]

    serie_indice_agente=datos_trm[datos_trm['date']>=rango_reg_agente[0]]
    serie_indice_agente=serie_indice_agente[serie_indice_agente['date']<=rango_reg_agente[1]]

    modelo_agente=SimpleExpSmoothing(serie_indice_agente['TRM']).fit(smoothing_level=0.5,optimized=False)
    prediccion_base_agente=modelo_agente.forecast()

    prediccion_base_agente=prediccion_base_agente.values[0]
    df_lecura_agente=pd.DataFrame({'index':agente['estrategiaLectura'] , 'impacto': agente['estrategiaLectura_imp']})

    lect_inicio=FECHA_PULSO-agente['numeroDiasLecturas']
    rango_lec_agente=[calendario.iloc[lect_inicio]['Date'],calendario.iloc[FECHA_PULSO]['Date']]

    serie_lectura_agente=datos_lectura[datos_lectura['date']>=rango_lec_agente[0]]
    serie_lectura_agente=serie_lectura_agente[serie_lectura_agente['date']<=rango_lec_agente[1]]
    serie_lectura_agente=serie_lectura_agente.merge(df_lecura_agente, left_on='index', right_on='index', how='left')

    serie_lectura_agente=serie_lectura_agente[serie_lectura_agente['impacto'].notnull()]
    serie_lectura_agente['factor']=serie_lectura_agente['polaridad']*serie_lectura_agente['impacto']
    
    #agente['pronosticos_dia_siguiente']=prediccion_base_agente*(1+serie_lectura_agente['factor'].mean()*agente['impactoTextos'])
    if len(serie_lectura_agente)>0 :
        #agente['pronosticos_dia_siguiente']=prediccion_base_agente*(1+serie_lectura_agente['factor'].mean()*agente['impactoTextos'])
        pronosticos_dia_siguiente=prediccion_base_agente*(1+serie_lectura_agente['factor'].mean()*agente['impactoTextos'])
    else:
        #agente['pronosticos_dia_siguiente']=prediccion_base_agente
        pronosticos_dia_siguiente=prediccion_base_agente
        
    return pronosticos_dia_siguiente

In [35]:
def activacionComprador(agente):
    #cuando
    agente['activo']=False
    #necesita stock?
    val_stock=agente['stockProducto']<agente['umbralStock']
    #el cambio del mercado le combiene?  prediccion - valor actual
    val_conveniencia=(agente['pronosticos_dia_siguiente'] - VALOR_MOMENTO_INDICE) > agente['umbralPronostico']
    #alguna de las 2 situaciones
    agente['activo']=val_stock or val_conveniencia
    #cuanto comprar
    agente['cantidadTransaccionar']=agente['capital']*agente['porcentajeCapitalCompra']
    #precio negociacion
    agente['precioNegociacion']=(agente['pronosticos_dia_siguiente'] - VALOR_MOMENTO_INDICE)*(1-agente['viveza'])+VALOR_MOMENTO_INDICE
    return agente['activo']

In [36]:
def activacionVendedor(agente):
    #cuando vender
    agente['activo']=False
    #necesita stock?
    val_stock=agente['stockProducto']>agente['umbralStock']
    #el cambio del mercado le combiene?  prediccion - valor actual
    val_conveniencia=(VALOR_MOMENTO_INDICE - agente['pronosticos_dia_siguiente'] ) > agente['umbralPronostico']
    #alguna de las 2 situaciones
    agente['activo']=val_stock or val_conveniencia
    #cuanto vender
    agente['cantidadTransaccionar']=agente['stockProducto']*agente['porcentajeStockVenta']
    #precio negociacion
    agente['precioNegociacion']=(agente['pronosticos_dia_siguiente'] - VALOR_MOMENTO_INDICE)*(1-agente['viveza'])+VALOR_MOMENTO_INDICE
    return agente['activo']

In [37]:
def transaccionarAgente(agenteComprador,agenteVendedor):
    #id agentes
    idComprador=agenteComprador.get('id')
    idVendedor=agenteVendedor.get('id')    
    #definir cantidad transaccionar
    cantidadRequeridaComprador=agenteComprador['cantidadTransaccionar']
    cantidadDisponibleVendedor=agenteVendedor['cantidadTransaccionar']
    cantidadTransaccion= cantidadRequeridaComprador if cantidadRequeridaComprador<cantidadDisponibleVendedor else cantidadDisponibleVendedor
    #precio transaccion
    precioTransaccion=agenteVendedor['precioNegociacion']
    #numeroPulsoActual
    #FECHA_PULSO
    
    #Cambio estado agentes
    #comprador
    agenteComprador['capital']=agenteComprador['capital']-precioTransaccion*cantidadTransaccion
    ##sin costo mauc
    agenteComprador['stockProducto']=agenteComprador['stockProducto']+cantidadTransaccion
    agenteComprador['cantidadTransaccionar']=agenteComprador['cantidadTransaccionar']-cantidadTransaccion
    #vendedor
    agenteVendedor['capital']=agenteVendedor['capital']+precioTransaccion*cantidadTransaccion
    agenteVendedor['stockProducto']=agenteVendedor['stockProducto']-cantidadTransaccion
    agenteVendedor['cantidadTransaccionar']=agenteVendedor['cantidadTransaccionar']-cantidadTransaccion
    #validar activacion agentes, retornar para acciones
    desactivarComprador=agenteComprador['cantidadTransaccionar']==0
    desactivarVendedor=agenteVendedor['cantidadTransaccionar']==0
    lineaTransaccion=[idComprador,idVendedor,cantidadTransaccion,precioTransaccion,FECHA_PULSO]
    #return {'desact_comp':desactivarComprador, 'desact_vend':desactivarVendedor, 'lineaTransaccion':lineaTransaccion}
    return [desactivarComprador, desactivarVendedor, lineaTransaccion]

In [38]:
def vivirComprador(agente):
    #aprovecha unidades
    unidadesAprovechadas=agente['habilidadAdministrativa']*agente['stockProducto']
    agente['stockProducto']=agente['stockProducto']-unidadesAprovechadas
    #tiene gastos
    agente['capital']=agente['capital']*(1-agente['controlGastosEImpuestos'])
    #recibe capital
    precioAprovechamiento=VALOR_MOMENTO_INDICE/(1-agente['rentabilidad'])
    agente['capital']=agente['capital']+unidadesAprovechadas*precioAprovechamiento
    #reevaluar costos fijos 
    #aumentar edad
    agente['edad']=agente['edad']+1

In [39]:
def vivirVendedor(agente):
    #tiene gastos
    agente['capital']=agente['capital']*(1-agente['controlGastosEImpuestos'])
    #le cuesta producir
    agente['capital']=agente['capital']*(1-agente['habilidadAdministrativa'])*agente['tasaProduccion']
    #aumenta el stock con lo producido
    agente['stockProducto']=agente['stockProducto']+agente['stockProducto']*agente['tasaProduccion']
    #aumenta edad
    agente['edad']=agente['edad']+1

# Universo

In [40]:
#GENERACION ESPONTANEA UNIVERSO
def generacionEspontanea():       
    for i in range(POBLACION_INICIAL):
        if(random.random()<=CANT_COMPRADORES):
            compradores.append(crearAgente(i,'Comprador'))
        else:
            vendedores.append(crearAgente(i,'Vendedor'))

In [41]:
def quiebraEmpresas():
    #quiebraEmpresas    
    for comp in compradores:
        capiltalEmpresa=comp['capital']
        valorStockEmpresa=comp['stockProducto']*VALOR_MOMENTO_INDICE
        valorEmpresa=capiltalEmpresa+valorStockEmpresa        
        #print("COMP capemp:",capiltalEmpresa,' valemp:',valorEmpresa, ' limQuiebra:',LIMITE_QUIEBRA)
        if capiltalEmpresa<LIMITE_QUIEBRA or valorEmpresa<LIMITE_QUIEBRA :
            #print("entraComp")
            compradores.remove(comp)     
            
    for vend in vendedores:
        capiltalEmpresa=vend['capital']
        valorStockEmpresa=vend['stockProducto']*VALOR_MOMENTO_INDICE
        valorEmpresa=capiltalEmpresa+valorStockEmpresa        
        #print("VEND capemp:",capiltalEmpresa,' valemp:',valorEmpresa, ' limQuiebra:',LIMITE_QUIEBRA)
        if capiltalEmpresa<LIMITE_QUIEBRA or valorEmpresa<LIMITE_QUIEBRA :
            #print("entraVend")
            vendedores.remove(vend)

In [42]:
def nuevosAgentes():
    #num_nuevos_agentes=math.floor(random.uniform(0, MAX_GENERACION_AGENTES)*POBLACION_ACTUAL)
    num_nuevos_agentes=math.floor(random.uniform(0, MAX_GENERACION_AGENTES)*POBLACION_ACTUAL)
    print("max_generacion_agentes: ",MAX_GENERACION_AGENTES,"nuevos Agentes: ", num_nuevos_agentes, "pob actual: ",POBLACION_ACTUAL, "pob final: ",POBLACION_ACTUAL+num_nuevos_agentes)
    for i in range(POBLACION_ACTUAL,POBLACION_ACTUAL+num_nuevos_agentes):
        if(random.random()<=CANT_COMPRADORES):
            compradores.append(crearAgente(i,'Comprador'))
        else:
            vendedores.append(crearAgente(i,'Vendedor'))

In [43]:
def pronosticoUniverso():
    for comp in compradores:
        #print(pronosticoAgente(comp))
        comp['pronosticos_dia_siguiente']=pronosticoAgente(comp)
    for vend in vendedores:
        #print(pronosticoAgente(vend))
        vend['pronosticos_dia_siguiente']=pronosticoAgente(vend)

In [44]:
def activacionUniverso():
    for comp in compradores:
        #print(pronosticoAgente(comp))
        auxActivacion=activacionComprador(comp)
        if auxActivacion:
            compradoresActivos.append(comp)
    for vend in vendedores:
        #print(pronosticoAgente(vend))
        auxActivacion=activacionVendedor(vend)
        if auxActivacion:
            vendedoresActivos.append(vend)

In [45]:
### Función transaccionar
def transaccionarUniverso():
    random.shuffle(compradoresActivos)
    
    for comp in compradoresActivos:     
        if len(vendedoresActivos)==0:
            #print("no hay vendedores")
            break

        df_ofertas=pd.DataFrame.from_dict(vendedoresActivos).sort_values('precioNegociacion')
        mejorVendedor=next(item for item in vendedoresActivos if item['id'] == df_ofertas.iloc[0]['id'])
        compradorMomento=comp

        precioMaximoCompra=compradorMomento['precioNegociacion']
        precioMinimoVenta=mejorVendedor['precioNegociacion']
        #selecciona si puede transaccionar
        if precioMaximoCompra>precioMinimoVenta:
            #print("transaccionar")
            #puede desactivar algun vendedor, antes de pasar a las siguientes transacciones
            (desactivarComprador, desactivarVendedor, lineaTransaccion)=transaccionarAgente(compradorMomento,mejorVendedor)
            #agregar transaccion a arreglo de transacciones con datos principales
            if desactivarComprador:
                compradoresActivos.remove(compradorMomento)
            if desactivarVendedor:
                vendedoresActivos.remove(mejorVendedor) 
            #registra la transacción
            DF_TRANSACCIONES.loc[len(DF_TRANSACCIONES)]=lineaTransaccion

In [46]:
def vivirUniverso():
    for comp in compradores:
        vivirComprador(comp)
    for vend in vendedores:
        vivirVendedor(vend)

## Pulso

In [47]:
compradores=[]
vendedores=[] 

In [48]:
generacionEspontanea()

In [49]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  73 numVendedores:  27


In [50]:
POBLACION_ACTUAL=len(compradores)+len(vendedores)

In [51]:
quiebraEmpresas()

In [52]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  72 numVendedores:  26


In [53]:
nuevosAgentes()

max_generacion_agentes:  0.2 nuevos Agentes:  13 pob actual:  100 pob final:  113


In [54]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  84 numVendedores:  27


In [55]:
pronosticoUniverso()

In [56]:
compradoresActivos=[]
vendedoresActivos=[]

In [57]:
activacionUniverso()

In [58]:
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))

numCompradoresActivos:  54 numVendedoresActivos:  22


In [59]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  84 numVendedores:  27


In [60]:
transaccionarUniverso()

In [61]:
DF_TRANSACCIONES

,idComprador,idVendedor,cantidadTransaccion,precioTransaccion,fechaPulso
0,109.0,58.0,263628.504317,3117.243382,36.0
1,46.0,48.0,161585.179686,3501.493527,36.0
2,66.0,108.0,159013.727612,3506.875788,36.0
3,8.0,59.0,762275.455798,3554.727531,36.0
4,77.0,35.0,461621.000487,3597.677243,36.0
5,41.0,13.0,36098.175067,3701.134757,36.0
6,84.0,72.0,35003.175035,3745.955734,36.0
7,20.0,90.0,248170.798587,3749.318776,36.0
8,39.0,26.0,337635.445913,3789.858485,36.0
9,67.0,4.0,210550.432744,3808.870645,36.0


In [62]:
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))

numCompradoresActivos:  54 numVendedoresActivos:  0


In [63]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  84 numVendedores:  27


In [64]:
vivirUniverso()

#### Fin Ciclo

# CICLO 2

# Preparar cambios siguiente ciclo

In [65]:
FECHA_PULSO=FECHA_PULSO+1
VALOR_MOMENTO_INDICE=datos_trm[datos_trm['date']<=calendario.iloc[FECHA_PULSO]['Date']].iloc[-1]['TRM']

In [66]:
POBLACION_ACTUAL=len(compradores)+len(vendedores)

In [67]:
POBLACION_ACTUAL

111

In [68]:
VALOR_MOMENTO_INDICE

3846.48

In [69]:
quiebraEmpresas()# capturar información de agentes quebrados
POBLACION_ACTUAL=len(compradores)+len(vendedores)
nuevosAgentes()
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))
pronosticoUniverso()
compradoresActivos=[]
vendedoresActivos=[]
activacionUniverso()
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))
transaccionarUniverso()
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))
vivirUniverso()
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

max_generacion_agentes:  0.2 nuevos Agentes:  8 pob actual:  107 pob final:  115
numCompradores:  87 numVendedores:  28
numCompradoresActivos:  68 numVendedoresActivos:  21
numCompradoresActivos:  68 numVendedoresActivos:  0
numCompradores:  87 numVendedores:  28


In [70]:
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

numCompradores:  87 numVendedores:  28


# CICLOS SIGUIENTES

In [84]:
FECHA_PULSO=FECHA_PULSO+1
VALOR_MOMENTO_INDICE=datos_trm[datos_trm['date']<=calendario.iloc[FECHA_PULSO]['Date']].iloc[-1]['TRM']
POBLACION_ACTUAL=len(compradores)+len(vendedores)
print("PULSO: ",FECHA_PULSO)
quiebraEmpresas()# capturar información de agentes quebrados
POBLACION_ACTUAL=len(compradores)+len(vendedores)
nuevosAgentes()
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))
pronosticoUniverso()
compradoresActivos=[]
vendedoresActivos=[]
activacionUniverso()
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))
transaccionarUniverso()
print("numCompradoresActivos: ",len(compradoresActivos), "numVendedoresActivos: ",len(vendedoresActivos))
vivirUniverso()
print("numCompradores: ",len(compradores), "numVendedores: ",len(vendedores))

PULSO:  51
max_generacion_agentes:  0.2 nuevos Agentes:  28 pob actual:  214 pob final:  242
numCompradores:  203 numVendedores:  39
numCompradoresActivos:  178 numVendedoresActivos:  31
numCompradoresActivos:  177 numVendedoresActivos:  0
numCompradores:  203 numVendedores:  39


In [85]:
DF_TRANSACCIONES

,idComprador,idVendedor,cantidadTransaccion,precioTransaccion,fechaPulso
0,109.0,58.0,2.636285e+05,3117.243382,36.0
1,46.0,48.0,1.615852e+05,3501.493527,36.0
2,66.0,108.0,1.590137e+05,3506.875788,36.0
3,8.0,59.0,7.622755e+05,3554.727531,36.0
4,77.0,35.0,4.616210e+05,3597.677243,36.0
...,...,...,...,...,...
356,190.0,94.0,2.015247e+05,3982.940654,51.0
357,167.0,163.0,2.278747e+05,4053.859871,51.0
358,134.0,235.0,2.789488e+04,4113.590030,51.0
359,172.0,138.0,8.078434e+04,4143.711819,51.0
